# Pre-trained YOLOv8 Model Integration

This notebook demonstrates how to load and use pre-trained YOLOv8 models for object detection using the Ultralytics implementation. We'll configure the models for inference and ensure they're ready for various applications.

**Contents:**
1. Setting up the environment
2. Loading pre-trained YOLOv8 models
3. Understanding model architecture and properties
4. Basic inference with sample images
5. Adjusting detection parameters
6. Exporting models to different formats

## 1. Setting up the Environment

First, let's install the necessary packages for working with YOLOv8. The primary package is `ultralytics`, which provides the YOLOv8 implementation.

In [ ]:
# Check if running in Colab or Kaggle
import sys
IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = 'kaggle_secrets' in sys.modules

if IN_COLAB or IN_KAGGLE:
    print(f"Running in {'Google Colab' if IN_COLAB else 'Kaggle'}, installing dependencies...")
    !pip install -q ultralytics
    !pip install -q opencv-python matplotlib
    !pip install -q torch torchvision
    !pip install -q ipywidgets
else:
    print("Not running in Colab or Kaggle. If needed, install dependencies manually.")

In [ ]:
# Import necessary libraries
import torch
import torchvision
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import os
from pathlib import Path
import time

# Set up matplotlib for inline plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]

In [ ]:
# Check system info and GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
    print(f"Using device: {device}")
else:
    device = torch.device("cpu")
    print(f"Using device: {device}")

## 2. Loading Pre-trained YOLOv8 Models

YOLOv8 comes with several pre-trained models of different sizes, optimized for different scenarios:

- **YOLOv8n**: Nano model (smallest, fastest, least accurate)
- **YOLOv8s**: Small model
- **YOLOv8m**: Medium model
- **YOLOv8l**: Large model
- **YOLOv8x**: Extra-large model (largest, slowest, most accurate)

Let's load a few of these models to see how they compare.

In [ ]:
# Load the YOLOv8 nano model
model_nano = YOLO('yolov8n.pt')

# Print model information
print(f"Model: YOLOv8n")
print(f"Task: {model_nano.task}")
print(f"Number of classes: {len(model_nano.names)}")
print(f"Input size: {model_nano.model.args['imgsz']}")

In [ ]:
# View model class names
print("COCO Classes detected by YOLOv8:")
for idx, (class_id, class_name) in enumerate(model_nano.names.items()):
    print(f"{class_id}: {class_name}", end="\t")
    if (idx + 1) % 5 == 0:  # Print 5 classes per line
        print()

In [ ]:
# Function to load different YOLOv8 models
def load_yolo_model(model_size='n'):
    """Load a YOLOv8 model of specified size
    
    Args:
        model_size (str): Model size (n, s, m, l, x)
        
    Returns:
        YOLO model instance
    """
    model_path = f'yolov8{model_size}.pt'
    model = YOLO(model_path)
    return model

In [ ]:
# Let's compare the model sizes
model_sizes = ['n', 's', 'm']
models = {}

print("Loading models of different sizes...")
for size in model_sizes:
    print(f"Loading YOLOv8{size}...")
    models[size] = load_yolo_model(size)
    
print("\nModel comparison:")
for size in model_sizes:
    model = models[size]
    # Count model parameters
    num_params = sum(p.numel() for p in model.model.parameters())
    print(f"YOLOv8{size}: {num_params/1e6:.2f} million parameters")

## 3. Understanding Model Architecture and Properties

Let's examine the YOLOv8 model architecture to understand how it works under the hood. YOLOv8 consists of:

1. **Backbone**: Extracts features from the input image (CSPDarknet based)
2. **Neck**: Aggregates features from different levels (Feature Pyramid Network)
3. **Head**: Makes predictions (bounding boxes, class probabilities, objectness scores)

In [ ]:
# Examine model structure - this will show the model summary with layer information
model = models['n']  # Use the nano model for illustration
print(model.model.info())

In [ ]:
# Analyze the model's PyTorch layers
def analyze_model_layers(model):
    """Analyze and print information about model layers"""
    # Get the underlying PyTorch model
    torch_model = model.model
    
    # Count different types of layers
    layer_counts = {}
    total_params = 0
    trainable_params = 0
    
    for name, module in torch_model.named_modules():
        class_name = module.__class__.__name__
        if class_name in layer_counts:
            layer_counts[class_name] += 1
        else:
            layer_counts[class_name] = 1
        
        # Count parameters
        for param in module.parameters(recurse=False):
            total_params += param.numel()
            if param.requires_grad:
                trainable_params += param.numel()
    
    print(f"Model Layer Analysis:\n")
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"\nLayer types:")
    for layer_type, count in sorted(layer_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"  {layer_type}: {count}")

# Run the analysis on the nano model
analyze_model_layers(model)

YOLOv8 uses a modular architecture with components like:

- **Conv**: Standard convolutional layers for feature extraction
- **C2f**: Cross-stage partial bottleneck with 2 convolutions and faster inference
- **SPPF**: Spatial Pyramid Pooling - Fast layer for extracting features at different scales
- **Detect**: Detection head that produces the final predictions

Each model size (n, s, m, l, x) uses the same architecture but with different scaling factors for width and depth.

## 4. Basic Inference with Sample Images

Now let's perform object detection using the pre-trained YOLOv8 model on some sample images.

In [ ]:
# Create a directory for saving test images if it doesn't exist
os.makedirs('test_images', exist_ok=True)

# Function to download an image from URL
def download_image(url, save_path=None):
    """Download an image from URL and return it as a numpy array"""
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    
    # Save image if path is provided
    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        img.save(save_path)
        print(f"Image saved to {save_path}")
    
    # Convert PIL image to numpy array
    img_array = np.array(img)
    
    # Convert RGB to BGR for OpenCV compatibility
    if len(img_array.shape) == 3 and img_array.shape[2] == 3:
        img_array = img_array[:, :, ::-1]
    
    return img_array

In [ ]:
# Sample image URLs for testing
test_images = [
    {'name': 'street_scene', 'url': 'https://ultralytics.com/images/zidane.jpg'},
    {'name': 'bus', 'url': 'https://ultralytics.com/images/bus.jpg'},
    {'name': 'people', 'url': 'https://raw.githubusercontent.com/ultralytics/assets/main/im/image2.jpg'}
]

# Download test images
image_paths = []
for img_info in test_images:
    image_path = f"test_images/{img_info['name']}.jpg"
    download_image(img_info['url'], image_path)
    image_paths.append(image_path)

In [ ]:
# Perform inference on the test images
model = models['n']  # Use the nano model for inference

# List to store detection results
results = []

# Process each image
for image_path in image_paths:
    # Run inference
    result = model(image_path)
    results.append(result[0])  # First element is the result for the first image
    
    # Print detection summary
    print(f"\nDetections in {image_path}:")
    boxes = result[0].boxes
    for i, box in enumerate(boxes):
        class_id = int(box.cls.item())
        class_name = model.names[class_id]
        confidence = box.conf.item()
        bbox = box.xyxy[0].tolist()  # xyxy format is [x1, y1, x2, y2]
        
        print(f"  {i+1}. {class_name} (Confidence: {confidence:.2f})")
        print(f"     Bounding box: {[round(coord, 2) for coord in bbox]}")

In [ ]:
# Visualize detection results
plt.figure(figsize=(16, 16))

for i, result in enumerate(results):
    # Create a subplot
    plt.subplot(len(results), 1, i+1)
    
    # Plot the result
    im_array = result.plot()
    plt.imshow(cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB))
    plt.title(f"Detection Results: {image_paths[i]}")
    plt.axis('off')
    
plt.tight_layout()
plt.show()

## 5. Adjusting Detection Parameters

YOLOv8 models allow us to adjust various parameters to tune the detection process. Let's explore some key parameters:

In [ ]:
# Function to perform inference with different confidence thresholds
def inference_with_conf_threshold(model, image_path, conf_thresholds=[0.25, 0.5, 0.75]):
    """
    Perform inference with different confidence thresholds and visualize results
    
    Args:
        model: YOLOv8 model
        image_path: Path to the input image
        conf_thresholds: List of confidence thresholds to test
    """
    plt.figure(figsize=(18, 5 * len(conf_thresholds)))
    
    for i, conf in enumerate(conf_thresholds):
        # Run inference with specific confidence threshold
        result = model(image_path, conf=conf)[0]
        
        # Plot result
        plt.subplot(len(conf_thresholds), 1, i+1)
        im_array = result.plot()
        plt.imshow(cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB))
        
        # Calculate number of detections
        num_detections = len(result.boxes)
        plt.title(f"Confidence Threshold: {conf} | Detections: {num_detections}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Test different confidence thresholds on one of the images
inference_with_conf_threshold(model, image_paths[0], [0.1, 0.25, 0.5, 0.75])

In [ ]:
# Function to perform inference with different IoU thresholds
def inference_with_iou_threshold(model, image_path, iou_thresholds=[0.45, 0.7, 0.9]):
    """
    Perform inference with different IoU thresholds and visualize results
    
    Args:
        model: YOLOv8 model
        image_path: Path to the input image
        iou_thresholds: List of IoU thresholds to test
    """
    plt.figure(figsize=(18, 5 * len(iou_thresholds)))
    
    for i, iou in enumerate(iou_thresholds):
        # Run inference with specific IoU threshold
        result = model(image_path, iou=iou)[0]
        
        # Plot result
        plt.subplot(len(iou_thresholds), 1, i+1)
        im_array = result.plot()
        plt.imshow(cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB))
        
        # Calculate number of detections
        num_detections = len(result.boxes)
        plt.title(f"IoU Threshold: {iou} | Detections: {num_detections}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Test different IoU thresholds on one of the images
inference_with_iou_threshold(model, image_paths[1], [0.25, 0.45, 0.75])

In [ ]:
# Function to compare different model sizes on the same image
def compare_model_sizes(models, image_path):
    """
    Compare detection results from different model sizes on the same image
    
    Args:
        models: Dictionary of YOLOv8 models of different sizes
        image_path: Path to the input image
    """
    plt.figure(figsize=(18, 5 * len(models)))
    
    for i, (size, model) in enumerate(models.items()):
        # Measure inference time
        start_time = time.time()
        result = model(image_path)[0]
        inference_time = time.time() - start_time
        
        # Plot result
        plt.subplot(len(models), 1, i+1)
        im_array = result.plot()
        plt.imshow(cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB))
        
        # Calculate number of detections
        num_detections = len(result.boxes)
        plt.title(f"YOLOv8{size} | Detections: {num_detections} | Inference time: {inference_time:.3f}s")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Compare different model sizes on one of the images
compare_model_sizes(models, image_paths[2])

## 6. Exporting Models to Different Formats

YOLOv8 models can be exported to various formats for deployment on different platforms. The Ultralytics framework makes this process straightforward.

In [ ]:
# Create directory for exported models
os.makedirs('exported_models', exist_ok=True)

# Define a function to export models
def export_model(model, format='onnx', imgsz=640):
    """
    Export YOLOv8 model to specified format
    
    Args:
        model: YOLOv8 model
        format: Export format (onnx, torchscript, openvino, etc.)
        imgsz: Image size for export
        
    Returns:
        Path to exported model
    """
    # Get model size from model
    model_type = model.ckpt_path.stem
    
    # Export the model
    export_path = f"exported_models/{model_type}_{format}"
    
    # Use the export method from the YOLO class
    result = model.export(format=format, imgsz=imgsz)
    
    return result

In [ ]:
# Export the nano model to ONNX format
try:
    export_model(models['n'], format='onnx')
    print("Model exported successfully to ONNX format")
except Exception as e:
    print(f"Error exporting model: {e}")

## 7. Creating a Reusable YOLOv8 Detector Class

Let's create a reusable class that encapsulates YOLOv8 functionality for easy use in other notebooks.

In [ ]:
class YOLOv8Detector:
    """A class for YOLOv8 object detection"""
    
    def __init__(self, model_size='n', conf=0.25, iou=0.45, device=None):
        """
        Initialize YOLOv8 detector
        
        Args:
            model_size (str): YOLOv8 model size ('n', 's', 'm', 'l', 'x')
            conf (float): Confidence threshold for detections
            iou (float): IoU threshold for NMS
            device (str): Device to use ('cuda' or 'cpu')
        """
        # Set device
        if device is None:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        else:
            self.device = device
            
        # Load model
        model_path = f'yolov8{model_size}.pt'
        self.model = YOLO(model_path)
        
        # Set detection parameters
        self.conf = conf
        self.iou = iou
        
        # Store class names
        self.class_names = self.model.names
        
        print(f"YOLOv8{model_size} detector initialized on {self.device}")
        print(f"Confidence threshold: {self.conf}, IoU threshold: {self.iou}")
        
    def detect(self, image_path, show_result=True):
        """
        Perform object detection on an image
        
        Args:
            image_path (str): Path to the input image
            show_result (bool): Whether to display the result
            
        Returns:
            Results object containing detections
        """
        # Run inference
        result = self.model(image_path, conf=self.conf, iou=self.iou)[0]
        
        # Display result if requested
        if show_result:
            im_array = result.plot()
            plt.figure(figsize=(12, 8))
            plt.imshow(cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.title(f"Detection Results: {Path(image_path).name}")
            plt.show()
            
            # Print detection summary
            print(f"\nDetections in {Path(image_path).name}:")
            boxes = result.boxes
            for i, box in enumerate(boxes):
                class_id = int(box.cls.item())
                class_name = self.class_names[class_id]
                confidence = box.conf.item()
                bbox = box.xyxy[0].tolist()  # xyxy format is [x1, y1, x2, y2]
                
                print(f"  {i+1}. {class_name} (Confidence: {confidence:.2f})")
        
        return result
    
    def detect_video(self, video_path=0, output_path=None, show_result=True):
        """
        Perform object detection on a video
        
        Args:
            video_path (str or int): Path to the input video or webcam index (0 for default camera)
            output_path (str): Path to save the output video
            show_result (bool): Whether to display the result in a notebook
            
        Returns:
            Path to the output video (if saved)
        """
        # Run inference
        results = self.model.predict(source=video_path, conf=self.conf, iou=self.iou, save=output_path is not None)
        
        # If we're saving the output, get the path
        if output_path is not None:
            output_path = results[0].save_dir
            print(f"Output video saved to {output_path}")
            
        # Return the output path if saved
        return output_path if output_path is not None else None
    
    def export(self, format='onnx', imgsz=640):
        """
        Export the model to specified format
        
        Args:
            format (str): Export format
            imgsz (int): Image size for export
            
        Returns:
            Path to exported model
        """
        return self.model.export(format=format, imgsz=imgsz)

In [ ]:
# Create a YOLOv8 detector instance
detector = YOLOv8Detector(model_size='n', conf=0.25, iou=0.45)

In [ ]:
# Test the detector on a sample image
for image_path in image_paths:
    result = detector.detect(image_path)

## 8. Integrating with PyTorch

YOLOv8 is built on PyTorch, which means we can easily integrate it with other PyTorch components. Let's explore how to access the underlying PyTorch model for more advanced use cases.

In [ ]:
# Access the PyTorch model
pytorch_model = detector.model.model
print(f"Type of PyTorch model: {type(pytorch_model)}")

# List the top-level modules
print("\nTop-level modules:")
for name, module in pytorch_model.named_children():
    print(f"{name}: {type(module).__name__}")

In [ ]:
# Function to manually run inference using PyTorch directly
def run_inference_pytorch(model, image_path, conf_thresh=0.25, iou_thresh=0.45):
    """
    Run inference using PyTorch model directly
    
    Args:
        model: PyTorch model
        image_path: Path to input image
        conf_thresh: Confidence threshold
        iou_thresh: IoU threshold
        
    Returns:
        Processed image with detections
    """
    # Read and preprocess image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resize and normalize image
    input_size = 640
    img_resized = cv2.resize(img_rgb, (input_size, input_size))
    img_float = img_resized.astype(np.float32) / 255.0
    
    # Convert to PyTorch tensor
    x = torch.from_numpy(img_float).permute(2, 0, 1).unsqueeze(0)
    
    # To device
    device = next(model.parameters()).device
    x = x.to(device)
    
    # Get detector model from YOLOv8 model
    detector_model = model
    
    # Run inference
    with torch.no_grad():
        preds = detector_model(x)
    
    # Process predictions (simplified version)
    # In practice, you would need to handle NMS and other post-processing
    # Using the YOLO class is much easier for this
    print("Raw prediction shape:", preds[0].shape)
    
    # Let's use the higher-level function for visualization instead
    yolo = detector.model  # Use the YOLO model we initialized earlier
    result = yolo(image_path)[0]
    
    # Plot the result
    im_array = result.plot()
    return im_array

# Test the PyTorch inference function
try:
    im_result = run_inference_pytorch(pytorch_model, image_paths[0])
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(im_result, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(f"PyTorch Direct Inference Result")
    plt.show()
except Exception as e:
    print(f"Error running direct PyTorch inference: {e}")
    print("\nNote: Direct access to the PyTorch model requires understanding of YOLOv8's internal structure.")
    print("Using the high-level YOLO API is recommended for most use cases.")

## 9. Summary and Next Steps

In this notebook, we've explored how to load and use pre-trained YOLOv8 models for object detection. We've covered:

1. Setting up the environment for YOLOv8
2. Loading pre-trained models of different sizes
3. Understanding the model architecture
4. Performing basic inference on images
5. Adjusting detection parameters like confidence and IoU thresholds
6. Exporting models to different formats
7. Creating a reusable detector class
8. Integrating with PyTorch

### Next Steps

In the upcoming notebooks, we'll cover:

1. **Image Upload and Detection**: Creating a user interface for uploading images and performing object detection
2. **Real-time Object Detection**: Implementing real-time detection using webcam input
3. **Advanced Features**: Exploring additional YOLOv8 capabilities and optimizations

With the foundation established in this notebook, we're now ready to build more practical applications using YOLOv8!